In [1]:
import tkinter as tk
import json
window = tk.Tk()
window.title("Sudoku")
icon = tk.PhotoImage(file= "Icon.png")
window.iconphoto(True, icon)
import time 
import tkinter as tk

canvas = tk.Canvas(window, width=750, height=750, bg="white")
canvas.grid()
canvas_size = 750
case_taille = 50
sudoku_taille = 9 * case_taille
x_offset = (canvas_size - sudoku_taille) // 2
y_offset = (canvas_size - sudoku_taille) // 2

# Dessiner la grille Sudoku
for i in range(10):
    width = 3 if i % 3 == 0 else 1  # Lignes plus épaisses pour les blocs 3x3
    canvas.create_line(
        x_offset + i * case_taille, y_offset,
        x_offset + i * case_taille, y_offset + sudoku_taille,
        width=width
    )
    canvas.create_line(
        x_offset, y_offset + i * case_taille,
        x_offset + sudoku_taille, y_offset + i * case_taille,
        width=width
    )
def lignes_uniques(grille):
    for ligne in grille:  
        deja_vus = set()  
        for nombre in ligne:  
            if nombre in deja_vus:  
                return False  
            deja_vus.add(nombre)  
    return True  
def colonnes_uniques(grille): 
    for col in range(len(grille[0])): 
        deja_vus = set()
        for ligne in grille: 
            if ligne[col] in deja_vus: 
                return False 
            deja_vus.add(ligne[col])
    return True
def carres_uniques(grille): 
    taille = 3
    for i in range(0, len(grille), taille):
        for j in range(0, len(grille), taille):
            deja_vus = set()
            for k in range(taille):
                for l in range(taille):
                    nombre = grille[i + k][j + l]
                    if nombre in deja_vus:
                        return False
                    deja_vus.add(nombre)
    return True
def afficher_chiffre(x, y, chiffre):
    """Affiche un chiffre dans une case spécifique."""
    canvas.create_text(
        50 + x * case_taille + case_taille // 2,
        50 + y * case_taille + case_taille // 2,
        text=str(chiffre))
    
mistakes_label = tk.Label(window, text="mistakes :", font=("Tekton Pro", 16), bg="#047274", fg="#f2f2f2")
mistakes_label.place(relx=0.8, rely=0.15, anchor="center")

def mistakes():
    cpt=0
    if lignes_uniques==False or colonnes_uniques==False or carres_uniques==False :
        cpt+=1
        mistakes_label.config(text="mistakes:"+cpt)

class Chronometre:
    def __init__(self, window):
        self.window = window
        self.window.title("Chronomètre")
        
        # Variables pour le chronomètre
        self.defile = False
        self.time_ecoule = 0
        self.start_time = 0
        
        # Affichage du temps
        self.time_affiche = tk.Label(self.window, text="0:00", font=("Helvetica", 48))
        self.time_affiche.grid()
        
        # Boutons Start/Stop et Reset
        self.start_stop_button = tk.Button(self.window, text="Démarrer", font=("Helvetica", 16), command=self.lancer)
        self.start_stop_button.grid(pady=10)
        
        self.reset_button = tk.Button(self.window, text="Réinitialiser", font=("Helvetica", 16), command=self.reinitialiser)
        self.reset_button.grid(pady=10)
        
        # Mise à jour du chronomètre
        self.update_timer()
    
    def lancer(self):
        """Démarre ou arrête le chronomètre"""
        if self.defile:
            self.defile = False
            self.start_stop_button.config(text="Démarrer")
        else:
            self.defile = True
            self.start_time = time.time() - self.time_ecoule  # Reprend à partir du temps écoulé
            self.start_stop_button.config(text="Arrêter")
    
    def reinitialiser(self):
        self.defile = False
        self.time_ecoule = 0
        self.time_affiche.config(text="0:00")
        self.start_stop_button.config(text="Démarrer")
    
    def update_timer(self):
        """Met à jour l'affichage du chronomètre"""
        if self.defile:
            self.time_ecoule = time.time() - self.start_time
        minutes = int(self.time_ecoule // 60)
        seconds = int(self.time_ecoule % 60)
        self.time_affiche.config(text=f"{minutes}:{seconds:02d}")
        
        # Mettre à jour toutes les 100 ms
        self.window.after(100, self.update_timer)


# Créer un objet Chronomètre
chronometre = Chronometre(window)
grille = {}
def sauvegarder_grille():
    with open("sudoku_sauvegarde.json", "w") as fichier:
        json.dump(grille, fichier)  # Enregistrer la grille dans un fichier JSON
    print("Grille sauvegardée avec succès.")
def reprendre_grille():
    global grille  # Permet de modifier la variable globale
    try:
        with open("sudoku_sauvegarde.json", "r") as fichier:
            grille = json.load(fichier)  # Charger la grille sauvegardée
        
        # Effacer l'affichage et re-dessiner la grille avec les valeurs sauvegardées
        canvas.delete("all")
        dessiner_grille()
        for key, value in grille.items():
            x, y = map(int, key.strip("()").split(","))  # Convertir la clé string en tuple (x, y)
            afficher_chiffre(x, y, value)
        
        print("Grille chargée avec succès.")
    except FileNotFoundError:
        print("Aucune sauvegarde trouvée.")
window.mainloop()